In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd './drive/My Drive/AI VIET NAM/Tensorflow/Generatives'


/content/drive/My Drive/AI VIET NAM/Tensorflow/Generatives


In [3]:
import time
import os
import sys
import tensorflow as tf
from PIL import Image
from collections import OrderedDict
from matplotlib.pyplot import imshow
import numpy as np
from scipy import interpolate
from scipy import misc
import glob
from shutil import copyfile
import imutils
from tensorflow.keras.applications import VGG19
import cv2
from tensorflow.keras.models import Model

model_dir = os.getcwd() + '/Models/'
content_dir = './quantative_style/Baed_E_scripts/content/' 
style_dir =  './quantative_style/Baed_E_scripts/50styles/'
Reference_dir = './quantative_style/Baed_E_scripts/content/'
source_dir = './NeuralStyleTransfer/outputs/'
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1', 'block5_conv1']
content_layers = ['block5_conv2'] 

#vgg definition that conveniently let's you grab the outputs from any layer
def vgg_layers(layer_names):
  vgg = VGG19(include_top=False, weights="imagenet")
  vgg.trainable= False
  outputs = [vgg.get_layer(name).output for name in layer_names]

  return Model([vgg.input], outputs)
    
    
    
# a function generate corvariant matrix and means with  input feature map
def cov_mean(input):
    b,h,w,c = input.shape
    F = tf.reshape(input,(b,-1,c))
    mean_ = tf.math.reduce_mean( F,axis=1, keepdims=True)
    mean = tf.concat(h*w*[mean_], axis=1)
    F = F-mean
    G = tf.matmul(tf.transpose(F, perm=[0,2,1]),F) 
    G = tf.math.divide(G, h*w)
    return tf.squeeze(G,axis=0), tf.squeeze(mean_)



    
# pre and post processing for images
#img_size = 512 
# prep = transforms.Compose([transforms.Scale(img_size),
#                            transforms.ToTensor(),
#                            transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to BGR
#                            transforms.Normalize(mean=[0.40760392, 0.45795686, 0.48501961], #subtract imagenet mean
#                                                 std=[1,1,1]),
#                            transforms.Lambda(lambda x: x.mul_(255)),
#                           ])
# postpa = transforms.Compose([transforms.Lambda(lambda x: x.mul_(1./255)),
#                            transforms.Normalize(mean=[-0.40760392, -0.45795686, -0.48501961], #add imagenet mean
#                                                 std=[1,1,1]),
#                            transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to RGB
#                            ])
# postpb = transforms.Compose([transforms.ToPILImage()])

# def postp(tensor): # to clip results in the range [0,1]
#     t = postpa(tensor)
#     t[t>1] = 1    
#     t[t<0] = 0
#     img = postpb(t)
#     return img

#get network
vgg = vgg_layers(style_layers)
# vgg.load_state_dict(torch.load(model_dir + 'vgg_conv.pth'))
# for param in vgg.parameters():
#     param.requires_grad = False
# if torch.cuda.is_available():
#     vgg.cuda()

    
references=[]
references = glob.glob(Reference_dir+"*.jpg")  # generate a list of reference images                    

Total_Covs=[0,0,0,0,0]
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1', 'block5_conv1'] 

for sample in references:    

    image = cv2.imread(sample)
    image = cv2.resize(image, (512,336))
    image = image/255.
    image = tf.cast(image, dtype=tf.float32)
    image = tf.expand_dims(image, axis=0)
    image = image *255.0
    image = tf.keras.applications.vgg19.preprocess_input(image)
    Covs_Means = [cov_mean(A) for A in vgg(image)] # generate corvariant matrix and means for each layer
    Total_Covs= [x+y[0] for x,y in zip(Total_Covs,Covs_Means)] # summation of corvariant matrix of each layer over references



80142336/80134624 [==============================] - 1s 0us/step


In [4]:
def prep(image):
    #image = cv2.imread(link)
    #image = cv2.resize(image, (512,336))
    image = image/255.
    image = tf.cast(image, dtype=tf.float32)
    image = tf.expand_dims(image, axis=0)
    image = image *255.0
    image = tf.keras.applications.vgg19.preprocess_input(image)
    return image

In [5]:
a = tf.random.normal((1,64,64,256))
b = tf.random.normal((1,4,9))

In [6]:
c,d= cov_mean(a)

In [7]:
c.shape, d.shape

(TensorShape([256, 256]), TensorShape([256]))

In [8]:
# Take an average over references

AVG_Covs = [x/len(references) for x in Total_Covs]

def PCA(A):
        S,U,V = tf.linalg.svd(A)
        return U,S,V 
# make a decomposition of each layer    
PCA_basis = [PCA(data) for data in AVG_Covs]


In [9]:
PCA_basis[0][2].shape

TensorShape([64, 64])

In [10]:
# Dimensions
Ks = [ 18,100,128,280,256]

file1 = open('./quantative_style/Baed_E_scripts/input.txt', 'r') 
file2= open('./quantative_style/Baed_E_scripts/EValuation3.txt', 'w')
columns = ["Name","E1","E2","E3","E4","E5","\n"]
name = '\t'.join(columns) 
file2.write(name)


for line in file1.readlines()[:]:
    print(line)
    
    filename  = line[:-1] 
    sp =line[:].split('_')
    style =int(sp[0][5:]) 
    content = int(sp[1][7:])
    method = sp[2]

    print(style,content,method,filename)
 

    img_dirs = [style_dir, source_dir+str(method)+'/']
    img_names = ['styles - %s.jpg'%style, filename]
    print(img_dirs[0] + img_names[0])
    imgs = [cv2.imread(img_dirs[i] + name) for i,name in enumerate(img_names)]
    imgs_torch = [prep(img) for img in imgs]
  
    # if torch.cuda.is_available():
    #     imgs_torch = [Variable(img.unsqueeze(0).cuda()) for img in imgs_torch]
    # else:
    #     imgs_torch = [Variable(img.unsqueeze(0)) for img in imgs_torch]
    style_image, syn_image= imgs_torch


    style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1', 'block5_conv1'] 
    content_layers = ['block5_conv2']  

    style_targets = [cov_mean(A) for A in vgg(style_image)]
    syn_results = [cov_mean(A) for A in vgg(syn_image)]


    def PCA_Proj(A,P,k):
        return tf.matmul(tf.matmul(tf.transpose(P[0][:,:k]),A[0]),P[2][:,:k]), tf.matmul( tf.expand_dims(A[1],axis=0), P[0][:,:k] ) 
    
    PCA_targets = [PCA_Proj(data,P ,k) for data,P,k in zip(style_targets,PCA_basis,Ks)]
    PCA_syn_results = [PCA_Proj(data,P ,k) for data,P,k in zip(syn_results,PCA_basis,Ks)]  
    #print(PCA_targets)   
  
    def Det(A,B):
        S,_,_ = tf.linalg.svd(A)
        #print(S)
        S1,_,_ = tf.linalg.svd(B)
        #print(S1)
        temp = tf.math.log(S1/S)
        #print(temp)
        u=0
        for a in temp:
            u +=a
        return u
    
    LogDet_AoverB = [ Det(syn[0],tar[0]) for syn,tar,k in zip(PCA_syn_results,PCA_targets,Ks)]
    #print(LogDet_AoverB[1])
    KLs = []

    KL_parts  = [ (tf.linalg.trace(tf.linalg.matmul( tf.linalg.inv(y[0]), x[0])).numpy(), tf.squeeze(tf.linalg.matmul( tf.linalg.matmul((y[1] -x[1]),  tf.linalg.inv(y[0])), tf.transpose((y[1]-x[1])) )).numpy() ,-k, logD.numpy()) for x,y,logD, k in zip(PCA_syn_results,PCA_targets,LogDet_AoverB,Ks)]
    print(KL_parts)
    KLs.append(np.sum(x) for x in KL_parts )  # np.sum(x) gives the 2 KL divergence
    #print(KLs[0])
    KL_array = np.array(KLs)
    #print(KL_array)
    Es = [ str(-np.log(x)+ np.log(2)) for x in KLs[0]] # E value is -log(KL)
    new = [str(filename)] + Es +["\n"]  
 
    name = '\t'.join(new) 
    file2.write(name)
file1.close()
file2.close()




style1_content123057_gramMatrix_iteration100.png

1 123057 gramMatrix style1_content123057_gramMatrix_iteration100.png
./quantative_style/Baed_E_scripts/50styles/styles - 1.jpg
[(46.001823, 2.5318742, -18, -11.128999), (313.31046, 13.299076, -100, -89.359085), (288.4136, 21.053715, -128, -74.73175), (499.66595, 20.720293, -280, -26.191702), (244.71643, 12.585899, -256, 227.72403)]
style1_content123057_gramMatrix_iteration1000.png

1 123057 gramMatrix style1_content123057_gramMatrix_iteration1000.png
./quantative_style/Baed_E_scripts/50styles/styles - 1.jpg
[(27.550812, 0.80336714, -18, -4.819681), (165.61072, 1.4210098, -100, -40.246), (210.65968, 4.182801, -128, -50.044044), (473.85092, 18.452326, -280, -45.902676), (222.59152, 17.993666, -256, 307.63733)]
style1_content123057_Pearson_iteration100.png

1 123057 Pearson style1_content123057_Pearson_iteration100.png
./quantative_style/Baed_E_scripts/50styles/styles - 1.jpg
[(62.688953, 3.8295662, -18, -16.591597), (367.67737, 36.345478,